Numbrix is a logic-based number-placement puzzle.[1]
The objective is to fill the grid so that each cell contains
digits in sequential order taking a horizontal or vertical
path; diagonal paths are not allowed. The puzzle setter
provides a grid often with the outer most cells completed.

Completed Numbrix puzzles are usually a square of numbers
in order from 1 to 64 (8x8 grid) or from 1 to 81 (9x9 grid),
following a continuous path in sequence.

The modern puzzle was invented by Marilyn vos Savant in 2008
and published by Parade Magazine under the name "Numbrix",
near her weekly Ask Marilyn article.

  1.  http://en.wikipedia.org/wiki/Numbrix

In [1]:
_ = 0
given = [[_,  _,  _,  _,  _,  _,  _,  _, _],
         [_, 11, 12, 15, 18, 21, 62, 61, _],
         [_,  6,  _,  _,  _,  _,  _, 60, _],
         [_, 33,  _,  _,  _,  _,  _, 57, _],
         [_, 32,  _,  _,  _,  _,  _, 56, _],
         [_, 37,  _,  _,  _,  _,  _, 73, _],
         [_, 38,  _,  _,  _,  _,  _, 72, _],
         [_, 43, 44, 47, 48, 51, 76, 77, _],
         [_,  _,  _,  _,  _,  _,  _,  _, _]]
n = len(given)

The given numbers are added as hard constraints to an indicator variable.

In [2]:
from minimum.linear.ip import IP
from minimum.linear.sum import Sum

ip = IP()
x = ip.add_boolean_cube(n, n, n*n)

for i in range(n):
    for j in range(n):
        if given[i][j] > 0:
            ip.add_constraint(x[i, j, given[i][j] - 1] == 1)

Every square contains exactly one number.

In [3]:
for i in range(n):
    for j in range(n):
        ip.add_constraint(sum(x[i, j, :]) == 1)

Every number occurs exactly once.

In [4]:
for k in range(n*n):
    ip.add_constraint(sum(sum(x[:, :, k])) == 1)

Every number (except the first) is neighbor to its predecessor.

In [5]:
for k in range(1, n*n):
    for i in range(n):
        for j in range(n):
            neighborhood = x[i, j, k]
            
            if i > 0:
                neighborhood -= x[i-1, j, k-1]
            if i < n - 1:
                neighborhood -= x[i+1, j, k-1]
            if j > 0:
                neighborhood -= x[i, j-1, k-1]
            if j < n - 1:
                neighborhood -= x[i, j+1, k-1]
            
            ip.add_constraint(neighborhood <= 0)

Solve the problem.

In [7]:
import minimum.linear.solver
minimum.linear.solver.Solver().solutions(ip).get()

v = []
for i in range(n):
    row = []
    for j in range(n):
        value = 0
        for k in range(n*n):
            value += (k+1) * x[i, j, k].value()
        row.append(Sum(value))
    v.append(row)

from minimum.linear.jupyter import display_2d
display_2d(v, color=given)